# Explore the dataset


In this notebook, we will perform an EDA (Exploratory Data Analysis) on the processed Waymo dataset (data in the `processed` folder). In the first part, you will create a function to display 

In [7]:
from utils import get_dataset

In [42]:
import os
import glob

import matplotlib
matplotlib.use('TkAgg')
            
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.patches import Rectangle
from PIL import Image

##### add
import cv2
import matplotlib.image as mpimg

In [9]:
paths = glob.glob('data/waymo/training_and_validation/*')
i = 0
#filename = os.path.basename(paths)
print(paths[i])
dataset = get_dataset(paths[i])
print('-----------------------------------------------------------')
print(dataset)

data/waymo/training_and_validation/segment-1051897962568538022_238_170_258_170_with_camera_labels.tfrecord
INFO:tensorflow:Reading unweighted datasets: ['data/waymo/training_and_validation/segment-1051897962568538022_238_170_258_170_with_camera_labels.tfrecord']
INFO:tensorflow:Reading record datasets for input file: ['data/waymo/training_and_validation/segment-1051897962568538022_238_170_258_170_with_camera_labels.tfrecord']
INFO:tensorflow:Number of filenames to read: 1
-----------------------------------------------------------
<DatasetV1Adapter shapes: {image: (None, None, 3), source_id: (), key: (), filename: (), groundtruth_image_confidences: (None,), groundtruth_verified_neg_classes: (None,), groundtruth_not_exhaustive_classes: (None,), groundtruth_boxes: (None, 4), groundtruth_area: (None,), groundtruth_is_crowd: (None,), groundtruth_difficult: (None,), groundtruth_group_of: (None,), groundtruth_weights: (None,), groundtruth_classes: (None,), groundtruth_image_classes: (None,),

print(dataset)

```
<
	DatasetV1Adapter shapes: 
	{
		image: (None, None, 3), 
		source_id: (), 
		key: (), 
		filename: (), 
		groundtruth_image_confidences: (None,), 
		groundtruth_verified_neg_classes: (None,), 
		groundtruth_not_exhaustive_classes: (None,), 
		groundtruth_boxes: (None, 4), 
		groundtruth_area: (None,), 
		groundtruth_is_crowd: (None,), 
		groundtruth_difficult: (None,), 
		groundtruth_group_of: (None,), 
		groundtruth_weights: (None,), 
		groundtruth_classes: (None,), 
		groundtruth_image_classes: (None,), 
		original_image_spatial_shape: (2,)
	}, 
	types: 
	{
		image: tf.uint8, 
		source_id: tf.string, 
		key: tf.string, 
		filename: tf.string, 
		groundtruth_image_confidences: tf.float32, 
		groundtruth_verified_neg_classes: tf.int64, 
		groundtruth_not_exhaustive_classes: tf.int64, 
		groundtruth_boxes: tf.float32, 
		groundtruth_area: tf.float32, 
		groundtruth_is_crowd: tf.bool, 
		groundtruth_difficult: tf.int64, 
		groundtruth_group_of: tf.bool, 
		groundtruth_weights: tf.float32, 
		groundtruth_classes: tf.int64, 
		groundtruth_image_classes: tf.int64, 
		original_image_spatial_shape: tf.int32
	}
>
```

## Write a function to display an image and the bounding boxes

Implement the `display_instances` function below. This function takes a batch as an input and display an image with its corresponding bounding boxes. The only requirement is that the classes should be color coded (eg, vehicles in red, pedestrians in blue, cyclist in green).

In [10]:
def display_instances(batch):
    """
    This function takes a batch from the dataset and display the image with 
    the associated bounding boxes.
    この関数は、データセットからバッチを取得し、関連する境界ボックスとともに画像を表示します。
    """
    # ADD CODE HERE

    ##### 色指定
    # color for different classes
#    colormap = {1: [1, 0 , 0], 2: [0, 0, 1], 4: [0, 1, 0]}
    colormap = {1:'blue', 2:'green', 4:'red'}
    
    ##### サブプロット領域設定。1×2、画像サイズ=(10, 10)
    num_col = 5
    num_row = (len(batch) + num_col -1) // num_col
    f, ax = plt.subplots(num_row, num_col, figsize=(20, 10))
    
    ##### batchのインデックスと中身を反復子にしてループ
    for idx, batch_data in enumerate(batch):
        img = batch_data["image"]
        x = idx // num_col
        y = idx % num_col       
        
        ##### 画像をセット
        ax[x, y].imshow(img)
        
        ##### バウンディボックス
        gt_boxes = batch_data["groundtruth_boxes"]
        gt_classes = batch_data["groundtruth_classes"]
        for bb, obj_class in zip(gt_boxes, gt_classes):
            y1, x1, y2, x2 = bb
            x1 *= img.shape[0]
            y1 *= img.shape[1]
            y2 *= img.shape[0]
            x2 *= img.shape[1]
            rec = Rectangle((x1, y1), x2-x1, y2-y1, facecolor='none', edgecolor=colormap[obj_class])
#            rec = Rectangle((x1, y1), x2-x1, y2-y1, facecolor='none', edgecolor='r')
            ax[x, y].add_patch(rec)
    plt.tight_layout()
    plt.show()

## Display 10 images 

Using the dataset created in the second cell and the function you just coded, display 10 random images with the associated bounding boxes. You can use the methods `take` and `shuffle` on the dataset.

In [11]:
## STUDENT SOLUTION HERE

batch = dataset.shuffle(10).take(10)
display_instances(list(batch.as_numpy_iterator()))

## Additional EDA

In this last part, you are free to perform any additional analysis of the dataset. What else would like to know about the data?
For example, think about data distribution. So far, you have only looked at a single file...

In [49]:
#batch = dataset.shuffle(10).take(10)

##### 画像群取得
def get_images(batch):    
    images = []
    for idx, batch_data in enumerate(batch):
        img = batch_data["image"]
        images.append(img)
    return images
    
images = get_images(batch)

##### kakuninn
#plt.imshow(images[0])
#plt.show()
print(type(images[0]))
print(len(images))
range = (0, 255)
#print(images)

<class 'tensorflow.python.framework.ops.EagerTensor'>
10


In [48]:
hsv_image = cv2.cvtColor(images[0], cv2.COLOR_RGB2HSV)

TypeError: Expected Ptr<cv::UMat> for argument 'src'

In [12]:
def get_cnt_class(dataset, num_dataset =100):
    gt_classes = np.array([])
    ##### 真値(gt)をバッチから抽出して classes に追加
    for batch in dataset.take(num_dataset):
#        gt_classes = np.append(gt_classes, batch["groundtruth_classes"].numpy())
        gt_classes.(batch["groundtruth_classes"].numpy())
    print(gt_classes)
    return np.unique(gt_classes, return_counts=True)

SyntaxError: invalid syntax (<ipython-input-12-8105773e2e59>, line 6)

In [ ]:
plt.rcParams['figure.figsize'] = [5, 5]
gt_classes, count = get_cnt_class(dataset, 100)

class_names = ['vehicle', 'pedestrian', 'cyclist']
plt.bar(left=range(len(classes)), height=count, tick_label=class_names)

In [46]:
#    for idx, batch_data in enumerate(batch):
#        img = batch_data["image"]


# ヒストグラム表示
def histogram(fn, images, range=(0, 255)):
#    images = [mpimg.imread(x) for x in images]
    images = [cv2.cvtColor(x, cv2.COLOR_RGB2BGR) for x in images]
    output = [fn(x) for x in images]
    plt.hist(output, range=range, bins=20)

def bright(img):
    return cv2.cvtColor(img, cv2.COLOR_BGR2HSV)[..., 2].mean()

def blue(img):
    return img[...,0].mean()

def green(img):
    return img[...,1].mean()

def red(img):
    return img[...,2].mean()

In [47]:
print(histogram(blue, images))

TypeError: Expected Ptr<cv::UMat> for argument 'src'

In [ ]:
histogram(green)

In [ ]:
histogram(red)

In [ ]:
histogram(bright)

In [11]:
##### 表示確認
# batch_data["groundtruth_classes"] in batch
#print(batch[0])
# print(batch["groundtruth_classes"] )

for idx, batch_data in enumerate(batch):
    print(batch_data)
#    gt_classes = batch_data["groundtruth_classes"].numpy()



{'image': <tf.Tensor: shape=(640, 640, 3), dtype=uint8, numpy=
array([[[190, 220, 255],
        [191, 221, 255],
        [191, 221, 255],
        ...,
        [ 56,  77,  80],
        [ 63,  81,  95],
        [ 48,  65,  85]],

       [[185, 215, 253],
        [187, 217, 253],
        [188, 218, 252],
        ...,
        [ 52,  73,  74],
        [114, 134, 145],
        [ 45,  64,  81]],

       [[179, 208, 248],
        [181, 211, 249],
        [183, 213, 249],
        ...,
        [ 44,  65,  66],
        [ 52,  72,  79],
        [ 46,  68,  79]],

       ...,

       [[ 86,  85,  93],
        [ 85,  84,  92],
        [ 83,  82,  90],
        ...,
        [ 62,  57,  64],
        [ 65,  59,  69],
        [ 57,  51,  61]],

       [[ 85,  85,  93],
        [ 84,  84,  92],
        [ 84,  84,  92],
        ...,
        [ 62,  54,  65],
        [ 60,  52,  63],
        [ 58,  50,  61]],

       [[ 82,  82,  90],
        [ 83,  83,  91],
        [ 84,  84,  92],
        ...,
        [ 6

In [16]:
##### カラーマップ
clr_map={1:'blue',2:'green',4:'red'}

##### ラベル設定
class_names = {1:'vehicle', 2:'pedestrian', 4:'cyclist'}

##### ラベル数カウント
batch = dataset.take(100)
##### 0:'vehicle', 1:'pedestrian', 2:'cyclist'
cnt_class = {1:0, 2:0, 4:0}

for idx, batch_data in enumerate(batch):
    gt_classes = batch_data["groundtruth_classes"]
    print(gt_classes)
    gt_classes = gt_classes.numpy()
    for gt_class in gt_classes:
        print(gt_class)
        cnt_class[gt_classes] += 1

print(cnt_class)

##### グラフ表示
plt.bar(class_names.values, cnt_class.values, clr_map.values,width=0.4)
plt.xlabel("class_names")
plt.ylabel("cnt_class")
plt.title("counts of object classes on waymo dataset")
plt.show()


tf.Tensor([1 1 1 1 1 1 1 1 2 2 1 1 2], shape=(13,), dtype=int64)
1


TypeError: unhashable type: 'numpy.ndarray'

In [19]:
import cv2

##### 画像群取得
images = []
for idx, batch_data in enumerate(batch):
    img = batch_data["image"]
    images.append(img)
range = (0, 255)
        
##### RGBのヒストグラム
red_means = []
blue_means = []
green_means = []
for img in images:
#    bgr_image = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    img = cv2.imread(img)
    red_means.append(img[:, :, 0])
    blue_means.append(img[:, :, 1])
    green_means.append(img[:, :, 2])
    
##### HSVのヒストグラム
hue_means = []
saturation_means = []
value_means = []
for img in images:
    img = cv2.imread(img)
    hsv_img = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    hue_means.append(img[:, :, 0])
    saturation_means.append(img[:, :, 1])
    value_means.append(img[:, :, 2])

TypeError: Expected Ptr<cv::UMat> for argument 'src'

In [20]:
plt.hist(red_means, range=range, bins=20)

ValueError: x must have 2 or fewer dimensions

In [ ]:
plt.hist(blue_means, range=range, bins=20)

In [ ]:
plt.hist(green_means, range=range, bins=20)

In [ ]:
plt.hist(hue_means, range=range, bins=20)

In [ ]:
plt.hist(saturation_means, range=range, bins=20)

In [ ]:
plt.hist(value_means, range=range, bins=20)